In [1]:
# crash colab to get more RAM
#!kill -9 -1

In [2]:
!nvidia-smi

Sun Aug  6 09:23:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/TIU-con/LED-Filtering

In [34]:
!pip install -U accelerate
! pip install -U transformers
!pip install accelerate -U


shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spi

In [6]:
!pip install datasets
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00


In [7]:
!python -m spacy download en_core_web_lg
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

2023-08-06 09:24:38.070027: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 09:24:39.074928: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-06 09:24:40.433942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-06 09:24:40.434415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [8]:
from datasets import load_dataset, load_metric
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re
nlp = spacy.load("en_core_sci_sm")

In [9]:
train_dataset_A = load_dataset("scientific_papers", "pubmed", split="train")
val_dataset_A = load_dataset("scientific_papers", "pubmed", split="validation")
test_dataset_A = load_dataset("scientific_papers", "pubmed", split="test")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [10]:
train_dataset_unfil = train_dataset_A.select(range(2000))
val_dataset_unfil = val_dataset_A.select(range(250))
test_dataset_unfil = test_dataset_A.select(range(250))

In [11]:
test_dataset_unfil

Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 250
})

In [12]:
from transformers import AutoTokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

In [14]:
# function to return a list of matched entities between abstract and source article

def entityMatch(ent, source):
  ent_split = ent.split()
  result = []
  for l in range(len(ent_split), 1, -1):
    for start_i in range(len(ent_split) - l + 1):
      sub_ent = " ".join(ent_split[start_i:start_i+l])
      if re.search(re.escape(sub_ent), source, re.IGNORECASE):
        result.append(sub_ent)
    if result:
      break
  if result:
    return result
  else:
    for token in ent_split:
      if token.lower() not in STOP_WORDS:
        if re.search(re.escape(token), source, re.IGNORECASE):
          result.append(token)
    return result
  #print([])
  return []

In [15]:
# entity-based data filtering
import csv
from datasets import Dataset
from sklearn import datasets
import pandas as pd
import datasets
def filter_dataset_by_precS(dataset):
    filtered_dataset = []
    for row in dataset:
      #print(type(row))
      art = nlp(row['article'])
      abs = nlp(row['abstract'])
      matched_entities = []
      entities_per_abs = []
      for e in abs.ents:
        entities_per_abs.append(e.text)
        match_result = entityMatch(e.text, row['article'])
        if match_result:
          matched_entities.append(match_result[0])
      precS = len(matched_entities) / len(entities_per_abs)
      if precS != 1:
        filtered_dataset.append(row)
    filtered_dataset1=datasets.Dataset.from_pandas(pd.DataFrame(filtered_dataset))

    return filtered_dataset1

In [16]:
filtered_test_DS=filter_dataset_by_precS(test_dataset_unfil)
#filtered_train_DS=filter_dataset_by_precS(train_dataset_unfil)
#filtered_val_DS=filter_dataset_by_precS(val_dataset_unfil)


In [17]:
len(filtered_test_DS)

240

In [ ]:
len(filtered_val_DS)

239

In [ ]:
len(filtered_train_DS)

1915

In [18]:
max_input_length = 8192
max_output_length = 512
batch_size = 2

In [19]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
filtered_train_DS

Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 1915
})

In [ ]:
train_dataset = filtered_train_DS.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map:   0%|          | 0/1915 [00:00<?, ? examples/s]

In [ ]:
val_dataset = filtered_val_DS.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

In [20]:
from transformers import AutoModelForSeq2SeqLM

In [21]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

In [22]:
# set generate hyperparameters
led.config.num_beams = 4
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [23]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [24]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="/content/drive/MyDrive/TIU-con/LED-Filtering",
    logging_steps=5,
    eval_steps=10,
    save_steps=200,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
)

In [ ]:
import accelerate
accelerate.__version__

'0.21.0'

In [25]:
from transformers import LEDTokenizer, LEDForConditionalGeneration
trainer = Seq2SeqTrainer(
    model=led,
    #model = LEDForConditionalGeneration.from_pretrained("/content/drive/MyDrive/TIU-con/TR/checkpoint-680").to("cuda").half(),
    #model=AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/TIU-con/TR/checkpoint-680", gradient_checkpointing=True, use_cache=False),
    tokenizer=tokenizer,
    args=training_args,
    #compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

NameError: ignored

In [ ]:
#Finetuning of filtered dataset with LED model using PubMed dataset(1915,239,240)
Trained_LED=trainer.train()

In [ ]:
#Explicitely you can save the finetuned model to your drive
trainer.save_model("/content/drive/MyDrive/TIU-con/LED-Filtering/Trainer")

In [ ]:
#automatically delete runtime
#from google.colab import runtime
#runtime.unassign()

## Evaluation

In [26]:
import torch

from datasets import load_dataset, load_metric
from transformers import LEDTokenizer, LEDForConditionalGeneration

# load pubmed
#pubmed_test = load_dataset("scientific_papers", "pubmed", ignore_verifications=True, split="test")

# load tokenizer
tokenizer = LEDTokenizer.from_pretrained("/content/drive/MyDrive/TIU-con/LED-Filtering/checkpoint-2200")
model = LEDForConditionalGeneration.from_pretrained("/content/drive/MyDrive/TIU-con/LED-Filtering/checkpoint-2200").to("cuda").half()
#model=Trained_LED

In [27]:
# function to make inference on a batch of dataset

def generate_answer(batch):
  inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=8192, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
  batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  return batch

In [28]:
# making inference on the without processed test dataset and generating summaries
result1 = filtered_test_DS.map(generate_answer, batched=True, batch_size=4)

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [29]:
# save to drive
import csv
result1.to_csv("/content/drive/MyDrive/TIU-con/LED-Filtering/LED_Filtering_result.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

4707275

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-149362a143f8>", line 3, in <cell line: 3>
    data = pd.read_csv("/content/drive/MyDrive/TIU-con/LED_Filtering/LED_Filtering_result.csv")
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 605, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", li

In [30]:
test_dataset = filtered_test_DS.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "section_names"],
)

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [31]:
# making inference on the processed filtered test dataset and generating summaries
result = test_dataset.map(generate_answer, batched=True, batch_size=4)

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [35]:
result

Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'global_attention_mask', 'labels', 'predicted_abstract'],
    num_rows: 240
})

In [ ]:
import socket
import threading

print("Server starts working")

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(("", 57054))
sock.listen(2)

conn, addr = sock.accept()


def get_message():
    while True:
        data = conn.recv(1024).decode()
        if len(data) != 0:
            print("Some guy: ", data)


def send_message():
    while True:
        message = input()
        if len(message) != 0:
            message = str.encode(message)
            conn.send(message)
            print("You: ", message)


def run():
    get_message_thread = threading.Thread(target=get_message)
    send_message_thread = threading.Thread(target=send_message)

    get_message_thread.daemon = True
    send_message_thread.daemon = True

    get_message_thread.start()
    send_message_thread.start()
    get_message_thread.join()
    send_message_thread.join()

run()

sock.close()

Server starts working


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-654df772c268>", line 10, in <cell line: 10>
    conn, addr = sock.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_

In [ ]:
#Another way check that
output=len(result)
with open('/content/drive/MyDrive/TIU-con/LED/test.csv', mode='w') as csv_file:
  fieldnames = ['Article', 'Actual Abstract','Predictive Abstract']
  writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
  writer.writeheader()
  for i in range(output):
    writer.writerow({'Article':result['article'][i],'Actual Abstract':result['abstract'][i],'Predictive Abstract':result['predicted_abstract'][i]})
#checking rows in file
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/TIU-con/LED/test.csv")
data